В рамках этого домашнего задания мы организовали для вас соревнование на площадке kaggle. <br><br>
Вам предстоит создать модель, которая делает максимально релевантные рекомендации фильмов для пользователей онлайн-кинотеатра. Вам доступна история просмотров, а также некоторая мета-информация о фильмах.  <br>
Наше соревнования на kaggle закрытое, к нему могут присоединиться только участники курса. Если вдруг вы еще не зарегистрировались на kaggle - самое время это сделать (kaggle.com). <br>
<br>
В рамках этого ноутбука вам предоставлен код для работы с данными, реализация целевой метрики (NDCG@10), а также реализована модель, рекомендующая топ фильмов из истории. <br>
Желаем успехов!

In [ ]:
from collections import Counter
import math

import pandas as pd
import numpy as np

import mmh3

from sklearn.utils import shuffle

import matplotlib.pyplot as plt
import seaborn 
from tqdm import tqdm_notebook
import random
import copy
seaborn.set()


Загрузим обучающие данные. 

In [ ]:
ratings = pd.read_csv('data/train.csv')

shape —  это свойство объекта DataFrame, которое позволяет узнать его размерность. Первое число —  количество строк в DataFrame, второе —  количеств столбцов (признаков)

In [ ]:
ratings.shape

head() - это функция объекта DataFrame, которая позволяет посмотреть на первые строчки датасета. 

In [ ]:
ratings.head()

Также вам предоставлены файлы с мета-информацией о фильмах и соответствующих тегах (data/recommendations/movies.csv, data/recommendations/tags.csv). Их можно использовать для вашей модели. 

In [ ]:
ll data/

In [ ]:
movies = pd.read_csv('data/movies.csv')

Тестовое множество в данном соревновании —  набор идентификаторов пользователей, для которых нужно сделать предсказания. Сохраним их в лист test_user_id. 

In [ ]:
with open('data/test_user_id.list', 'r') as file:
    test_user_id = file.read()
test_user_id = [int(user_id) for user_id in test_user_id.split(',')]

Выведем на экран первые 10 идентификаторов пользователей из тестового множества

In [ ]:
test_user_id[:10]

### Разделим данные на обучение и валидацию

Важная деталь! Так как данные в нашем датасете на самом деле отсортированы по времени, то корректное разбиение на обучение и валидацию будет следующим: необходимо в качестве обучающего множества брать наблюдения более ранние по времени, а для валидации —  наиболее поздние.

In [ ]:
train = ratings[:int(ratings.shape[0] * 0.75)]
validation = ratings[int(ratings.shape[0] * 0.75):]

Посмотрим на размерности получившихся множеств для обучения и валидации

In [ ]:
train.shape, validation.shape

### Реализация целевой метрики

Ваша задача —  предсказать **релевантность** фильмов тому или иному пользователю. Релевантность — это функция, характеризующая, насколько объект (в нашем случае —  фильм) подходит пользователю. Чем выше значение функции релевантности, тем больше фильм подходит пользователю. 

При этом, так как мы строим рекомендательную систему, наша задача — найти самые релевантные  для пользователя фильмы. Поэтому нам не важно абсолютное значение релевантности, а важен порядок, в соответствии с которым мы выстроили фильмы согласно функции релевантности. 

Такая задача (в которой нам важно упорядочить элементы) называется **задачей ранжировани**. Кроме рекомендательной системы, задачу ранжирования решает, например, поисковая система:  ее функция — упорядочить web-страницы в порядке релевантности запросу пользователя. 

Для того, чтобы измерить качество ранжирования, нам понадобится специальная метрика NDCG (Normalized Discounted Cummulative Gain). 
Она сравнивает порядок, в котором мы упорядочили фильмы, с идеальным порядком (при котором сначала стоят самые релевантные фильмы, а в конце — самые нерелевантные). Если элемент стоит дальше того места, где ему положено стоять, NDCG уменьшает свое значение. При этом она учитывает номер, на котором произошло нарушение порядка: понятно, что поменять местами 1-й и 10й фильмы намного хуже чем 101-й и 110-й. На первую рекомендацию пользователь посмотрит гарантированно (и там окажется элемент, который должен был быть 10м), а до 101-го с большой вероятностью просто не дойдет. 

Важность позиции в NDCG убывает в  соответствии с обратным логарифмическим законом. 

При использовании метрики NDCG@K, важность позиций с номером большим, чем K, полагается равной нулю.

Более подробно про метрику NDCG (с формулами!) можете почитать по ссылке: https://habr.com/company/econtenta/blog/303458/

In [ ]:
K = 30
max_n = 35

x = [i for i in range(1, max_n)]
y = [(i <= K) * 1/math.log2(i + 1) for i in range(1, max_n)]

plt.figure(figsize=(10, 6))
plt.title("Относительная важность ошибки на i-й позиции метрики NDCG@{}".format(K))
plt.xlabel("Номер позиции")
plt.ylabel("Относительная важность ошибки")
plt.text(5, 0.1, """после {}й позиции происходит резкий скачок в ноль,
так как метрика NDCG@{} считает все позиции > {} неважными""".format(K, K, K), bbox=dict(facecolor='white', alpha=0.5))

plt.plot(x, y);
plt.show();

In [ ]:
def dcg_at_k(r, k, method=0):
    """
    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)
        k: Number of results to consider
        method: If 0 then weights are [1.0, 1.0, 0.6309, 0.5, 0.4307, ...]
                If 1 then weights are [1.0, 0.6309, 0.5, 0.4307, ...]
    Returns:
        Discounted cumulative gain
    """
    r = np.asfarray(r)[:k]
    if r.size:
        if method == 0:
            return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
        elif method == 1:
            return np.sum(r / np.log2(np.arange(2, r.size + 2)))
        else:
            raise ValueError('method must be 0 or 1.')
    return 0.



def ndcg_at_k(r, k, method=0):
    """
    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)
        k: Number of results to consider
        method: If 0 then weights are [1.0, 1.0, 0.6309, 0.5, 0.4307, ...]
                If 1 then weights are [1.0, 0.6309, 0.5, 0.4307, ...]
    Returns:
        Normalized discounted cumulative gain
    """
    dcg_max = dcg_at_k(sorted(r, reverse=True), k, method)
    if not dcg_max:
        return 0.
    return dcg_at_k(r, k, method) / dcg_max

Примеры использования метрики. r — это лист, содержащий значение релевантности объекта, находящегося на этой позиции. Чем выше значение, тем больше релеватность. 0 — объект не релевантен.

In [ ]:
r = [3, 2, 3, 0, 0, 1, 2, 2, 3, 0]
ndcg_at_k(r, 1)

In [ ]:
r = [2, 1, 2, 0]
ndcg_at_k(r, 4)

In [ ]:
ndcg_at_k([0], 1)

In [ ]:
ndcg_at_k([1], 2)

Помните, что в задаче ранжирования важен сам факт того, что пользователь посмотрит фильм, и не важна его конкретная оценка. 

### Реализация топ-рекоммендера

In [ ]:
class TopRecommender(object):
    def fit(self, train_data):
        counts = Counter(train_data['movieId'])
        self.predictions = counts.most_common()
        
    def predict(self, user_id, n_recommendations=10):
        return [movie_id for movie_id, frequency in self.predictions[:n_recommendations]]

Построим модель и посмотрим на качество по метрике NDCG@10 на валидационном множестве

In [ ]:
recommender_train = TopRecommender()
recommender_train.fit(train)

In [ ]:
verbose = True
num_to_print = 10
total_ndcg = 0

for user_id, group in validation.groupby('userId'):
    ground_truth_films = [int(data.movieId) for row, data in group.iterrows()]
    recommendations = recommender_train.predict(user_id, n_recommendations=10)
    relevance_scores = []
    for rec in recommendations:
        if rec in ground_truth_films:
            relevance_scores.append(len(ground_truth_films) - ground_truth_films.index(rec))
        else:
            relevance_scores.append(0)
    total_ndcg += ndcg_at_k(relevance_scores, k=10)
    
    if verbose and np.random.random() > 0.999:
        user_films_train = train[train.userId == user_id].movieId.values
        print('Идентификатор пользователя: ', user_id)
        print(
            'Фильмы в обучающей выборке для этого пользователя:',
            [movies[movies.movieId == movie_id].title.values[0] for movie_id in user_films_train[:num_to_print]],
            '\n'
        )
        print(
            'Просмотренные на самом деле фильмы: ', 
            [movies[movies.movieId == movie_id].title.values[0] for movie_id in ground_truth_films[:num_to_print]],
            '\n'
        )
        print(
            'Рекомендации топ-рекомендера: ', 
            [movies[movies.movieId == rec_id].title.values[0] for rec_id in recommendations],
            '\n'
        )
        print('Значение NDCG@10 = ', ndcg_at_k(relevance_scores, k=10), '\n\n')

In [ ]:
total_ndcg / validation.shape[0]

### Обучим топ-рекоммендер на всем обучающем множестве

In [ ]:
recommender = TopRecommender()
recommender.fit(ratings)

Попробуем сделать предсказание для первого пользователя в тесте

In [ ]:
recommender.predict(user_id=test_user_id[0], n_recommendations=10)

### Создание файла с решением

Сформируем файл с решением, содержащим предсказания топ-рекоммендера

In [ ]:
with open('submit.csv', 'w') as f:
    f.write('userId,movieId\n')
    for user_id in test_user_id:
        recommendations = recommender.predict(user_id=user_id, n_recommendations=10)
        for rec in recommendations:
            f.write(str(user_id) + ',' + str(int(rec)) + '\n')
    print('Отлично! Время загрузить файл submit.csv на kaggle!')

### Наши советы: что можно пробовать в этом хакатоне

1) Загрузите и отправьте на kaggle результат TopRecommender-а

2) Имплементируйте SVD-рекоммендер (воспользовавшись примером из модуля)

3) Попробуйте поиграть параметрами SVD-рекоммендера (например, количеством скрытых параметров)

4) Подумайте, как скомбинировать вывод svd-рекоммендера с параметрами фильма

5) Lightgbm поддерживает прямую оптимизацию по NDCG


Не обязательно выполнять все эти пункты; также можно пробовать что-то другое, до чего еще никто не догадался.
Очень много материалов про обучение рекомендательных систем можно найти в интернете. 

Успехов!

# Разбор

Давайте для начала поймем, нужно ли нам решать проблему холодного старта. Для этого проверим, есть ли в списке контрольных пользователей пользователи, которых не было в наборе, выданном нам для обучения. 

Для этого создадим множества для каждого из наборов пользователей и проверим, есть ли пользователи в первом, которых нет во втором. 

In [ ]:
#Множество пользоватей из контрольного набора: 
control_users_set = set(test_user_id)

#Множество пользователей из выданного нам тренировочного набора:
ratings_user_set = set(ratings.userId)

In [ ]:
control_users_set - ratings_user_set

Отлично, значит в контрольном множестве содержатся только те же пользователи, что и в тренировочном. Это означает, что нам не надо заниматься решением проблемы "холодного старта" — пытаться предсказывать что-либо для пользователей, которых не было в тренировочном сете. 

Что ж, давайте для начала удалим из валидационного набора пользователей, которые не встречались в тренировочных данных:

In [ ]:

print("Количество пользователей в валидационной выборке до фильтрации", len(set(validation.userId)))

train_users_set = set(train.userId)
validation = validation[validation.userId.apply(lambda userId: userId in train_users_set)]
print("Количество пользователей в валидационной выборке после фильтрации", len(set(validation.userId)))


Для удобства давайте обернем наши вызовы для рассчета NDCG для рекоммендера и для формирования сабмита в специальные функции get_ndcg и make_submit. Это пригодится нам, когда мы будем перебирать гиперпараметры алгоритмов. Обратите внимание, что в отличие от примера выше,  мы делим total_ndcg на количество пользователей, а не на общее количество примеров в тестовой выборке. Данные значения отличаются только в константное количество раз, и при переборе гиперпараметров это не важно, однако NDCG — это метрика набора рекомендаций, поэтому правильно усреднять именно по количеству таких наборов, которое равно количеству пользователей в тестовом сете.

In [ ]:
def sample_users(dataset, fraction):
    all_users = list(set(dataset.userId))
    filtered_users = set(filter(lambda x: random.random() <= fraction, all_users))
    return dataset[dataset.userId.apply (lambda x: x in filtered_users)]

In [ ]:
def get_ndcg(recommender, validation_fraction=1.0):
    recommender.fit(train)
    total_ndcg = 0
    
    sampled_validation = sample_users(validation, validation_fraction)
    
    validation_by_user = sampled_validation.groupby('userId')
    for user_id, group in validation_by_user:
        ground_truth_films = [int(data.movieId) for row, data in group.iterrows()]
        recommendations = recommender.predict(user_id, n_recommendations=10)
        relevance_scores = []
        for rec in recommendations:
            if rec in ground_truth_films:
                relevance_scores.append(len(ground_truth_films) - ground_truth_films.index(rec))
            else:
                relevance_scores.append(0)
        total_ndcg += ndcg_at_k(relevance_scores, k=10)
    return total_ndcg/len(validation_by_user)


def make_submit(recommender, filename):
    recommender.fit(ratings)
    with open(filename, 'w') as f:
        f.write('userId,movieId\n')
        for user_id in test_user_id:
            recommendations = recommender.predict(user_id=user_id, n_recommendations=10)
            for rec in recommendations:
                f.write(str(user_id) + ',' + str(int(rec)) + '\n')

In [ ]:
recommender = TopRecommender()
print("topRecommender NDCG: ", get_ndcg(recommender))
make_submit(recommender, "top_recommender.csv")

На Kaggle TopRecommender набирает 0.02410 балла. Разница в показателях связана с тем, что мы отдали в "тест" много данных (25%), за это время "вкусы" пользователя могут поменяться и сдвинуться ближе к "среднему". Тем не менее, мы можем пользоваться данной оценкой для понимания "направления" правильности наших предсказаний. Теперь давайте попробуем улучшить качество предсказаний.

Для начала воспользуемся примером SVD-рекоммендера из лекций. Отличие будет заключаться в том, что в лекциях мы возвращали и фильм и его предсказанное значение релевантности; в данном случае будем возвращать только ID фильма.

In [ ]:
from collections import defaultdict
from scipy.sparse import csr_matrix
from sklearn.decomposition import TruncatedSVD


class SVDRecommender(object):
    def fit(self, data, n_components = 30):     
        self.users = defaultdict(lambda: len(self.users))
        self.movies = defaultdict(lambda: len(self.movies))
        rows = data.userId.apply(lambda userId: self.users[userId])
        cols = data.movieId.apply(lambda movieId: self.movies[movieId])
        vals = [1.0]* len(cols)
        self.interactions_matrix = csr_matrix((vals, (rows, cols)))
        self.model = TruncatedSVD(n_components = n_components, algorithm='arpack')
        self.model.fit(self.interactions_matrix)
        self.movies_reverse = {}
        for movie_id in self.movies:
            movie_idx = self.movies[movie_id]
            self.movies_reverse[movie_idx] = movie_id     

        
    def predict(self, user_id, n_recommendations=10):        
        user_interactions = self.interactions_matrix.getrow(self.users[user_id])    
        user_low_dimensions = self.model.transform(user_interactions)  
        return self.predict_low_dimension(user_low_dimensions, user_interactions, n_recommendations)
    
    def predict_low_dimension(self, user_low_dimensions, user_interactions, max_n=10):
        user_predictions = self.model.inverse_transform(user_low_dimensions)[0]
        recommendations = []
        
        for movie_idx in reversed(np.argsort(user_predictions)):
            #Добавляем фильм к рекомендациям только если пользователь его еще не смотрел
            if user_interactions[0, movie_idx] == 0.0:
                movie = self.movies_reverse[movie_idx]
                score = user_predictions[movie_idx]
                
                #Эта строчка отличается из примера из лекций
                recommendations.append(movie)
                
            if (len(recommendations) >= max_n):
                return recommendations

In [ ]:
get_ndcg(SVDRecommender())

Ух ты, в 2 раза лучше! Давайте сделаем сабмит.

In [ ]:
make_submit(SVDRecommender(), "svd_recommender.csv")

Просто SVD (скопированный из лекциии без изменений) набирает 0.0468 в kaggle. Этого достаточно, чтобы получить "Зачет". 

Давайте попробуем улучшить результат. Для этого воспользуемся поиском гиперпараметров для svd.
Посмотрим на описание SVD из sklearn(https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html) и увидим, что по сути мы можем влиять на 2 параметра: n_components и algorithm. 

Давайте для начала просто попробуем подобрать идеальное значение n_components. 

Для того, чтобы нам было удобнее выполнять перебор, вынесем параметры n_components и algorithm в конструктор объекта(функцию ```__init__```)

In [ ]:
class SVDRecommender(object):
    def __init__(self, n_components = 30, algorithm='arpack'):
        self.n_components = n_components
        self.algorithm = algorithm
    
    def fit(self, data):     
        self.users = defaultdict(lambda: len(self.users))
        self.movies = defaultdict(lambda: len(self.movies))
        rows = data.userId.apply(lambda userId: self.users[userId])
        cols = data.movieId.apply(lambda movieId: self.movies[movieId])
        vals = [1.0]* len(cols)
        self.interactions_matrix = csr_matrix((vals, (rows, cols)))
        self.model = TruncatedSVD(n_components=self.n_components, algorithm=self.algorithm)
        self.model.fit(self.interactions_matrix)
        self.movies_reverse = {}
        for movie_id in self.movies:
            movie_idx = self.movies[movie_id]
            self.movies_reverse[movie_idx] = movie_id     

        
    def predict(self, user_id, n_recommendations=10):        
        user_interactions = self.interactions_matrix.getrow(self.users[user_id])    
        user_low_dimensions = self.model.transform(user_interactions)  
        return self.predict_low_dimension(user_low_dimensions, user_interactions, n_recommendations)
    
    def predict_low_dimension(self, user_low_dimensions, user_interactions, max_n=10):
        user_predictions = self.model.inverse_transform(user_low_dimensions)[0]
        recommendations = []
        for movie_idx in reversed(np.argsort(user_predictions)):
            if user_interactions[0, movie_idx] == 0.0:
                movie = self.movies_reverse[movie_idx]
                score = user_predictions[movie_idx]
                recommendations.append(movie)
            if (len(recommendations) >= max_n):
                return recommendations

Для начала пробежимся "грубо", проверяя каждый 10й элемент, чтобы примерно представить, где нужно искать максимум качества. 

In [ ]:
x =[]
y = []

max_ndcg = 0.0
iteration = 0
for n_components in range(2, 200, 10):
    iteration += 1
    recommender = SVDRecommender(n_components=n_components)
    ndcg = get_ndcg(recommender)
    x.append(n_components)
    y.append(ndcg)
    
    if ndcg > max_ndcg: max_ndcg = ndcg
    print("iteration: {}, n_components: {}, NDCG: {}, max_ndcg:  {}".format(iteration, n_components, ndcg, max_ndcg))

In [ ]:
plt.plot(x, y)

Видно, что у параметра n_components есть 2 оптимума: в районе 22 и в районе 72 компонент. Давайте пробежимся в окрестностях каждого из них и попробуем найти максимумы. 
Попробуем отправить оба в kaggle

In [ ]:
make_submit(SVDRecommender(n_components=22), "svd22.txt")
make_submit(SVDRecommender(n_components=72), "svd72.txt")

SVD c 22 компонентами позволяет набрать 0.048 в kaggle.

Давайте подумаем вот о чем: интересы пользователя со временем могут меняться. Поэтому разумно взять не всю историю пользователя, 
а только N последних его фильмов. Давайте посмотрим, что из этого выйдет. 

In [ ]:
class SVDRecommenderKeepN(object):
    def __init__(self, n_components = 30, algorithm='arpack', keep_last_n=20):
        self.n_components = n_components
        self.algorithm = algorithm
        self.keep_last_n = keep_last_n
        self.user_seen = {}
    
    def fit(self, data):     
        self.users = defaultdict(lambda: len(self.users))
        self.movies = defaultdict(lambda: len(self.movies))
        rows = []
        cols = []
        for user_id, group in data.groupby('userId'):
            all_user_movies = [self.movies[int(d.movieId)] for row, d in group.iterrows()]
            self.user_seen[user_id] = set(all_user_movies)
            user_keep_movies = all_user_movies[-self.keep_last_n:]
            rows += [self.users[user_id]] * len(user_keep_movies)
            cols += user_keep_movies
        vals = [1.0]* len(cols)
        self.interactions_matrix = csr_matrix((vals, (rows, cols)))
        self.model = TruncatedSVD(n_components=self.n_components, algorithm=self.algorithm)
        self.model.fit(self.interactions_matrix)
        self.movies_reverse = {}
        for movie_id in self.movies:
            movie_idx = self.movies[movie_id]
            self.movies_reverse[movie_idx] = movie_id     

        
    def predict(self, user_id, n_recommendations=10):        
        user_interactions = self.interactions_matrix.getrow(self.users[user_id])    
        user_low_dimensions = self.model.transform(user_interactions)  
        return self.predict_low_dimension(user_low_dimensions, user_interactions, user_id, n_recommendations)
    
    def predict_low_dimension(self, user_low_dimensions, user_interactions, user_id, max_n=10):
        user_predictions = self.model.inverse_transform(user_low_dimensions)[0]
        recommendations = []
        for movie_idx in reversed(np.argsort(user_predictions)):
            if movie_idx in self.user_seen[user_id]:
                continue
            movie = self.movies_reverse[movie_idx]
            score = user_predictions[movie_idx]
            recommendations.append(movie)
            if (len(recommendations) >= max_n):
                return recommendations

In [ ]:
recommender = SVDRecommenderKeepN(n_components=22, keep_last_n=100)
get_ndcg(recommender)

In [ ]:
x =[]
y = []

max_ndcg = 0.0
iteration = 0


for keep_last_n in range(1, 25):
    iteration += 1
    recommender = SVDRecommenderKeepN(n_components=22, keep_last_n=keep_last_n)
    ndcg = get_ndcg(recommender)
    if ndcg > max_ndcg: max_ndcg = ndcg
    x.append(keep_last_n)
    y.append(ndcg)
    print("iteration: {}, keep_last_n: {}, NDCG: {}, max_ndcg:  {}".format(iteration, keep_last_n, ndcg, max_ndcg))
    

In [ ]:
plt.plot(x, y)

Очень интересно! Видим, что очень неплохо работает предсказание всего лишь по одному последнему фильму, дальше происходит резкий провал, а потом качество возвращается на исходную позицию только в районе 20 фильмов. Скорее всего так получается из-за того, что пользователи часто смотрят сиквел фильма после первой части (например, "Терминатор 2" после "Терминатор 1"), и в этом случае последовательность оказывается очень важна. 

Давайте отправим в kaggle оба варианта и посмотрим на результат. 

In [ ]:
make_submit(SVDRecommenderKeepN(n_components=22, keep_last_n=1), "svd_keep_1.csv")
make_submit(SVDRecommenderKeepN(n_components=22, keep_last_n=19), "svd_keep_19.csv")

Каждый из этих рекоммендеров по отдельности не позволяет нам получить результат лучше просто svd. Однако мы можем увидеть,
что для рекомендаций важны как долгосрочные, так и краткосрочные действия пользователя. Давайте попробуем восопользоваться ML-ем, чтобы скомбинировать их оптимальным образом. 

In [ ]:
from lightgbm import LGBMRanker
from collections import defaultdict
from scipy.sparse import csr_matrix
from sklearn.decomposition import TruncatedSVD

class MLRecommender(object):
    def __init__(self, target_movies = 3, 
                 svd_at=22,
                 ranking_samples=150,
                 keep_movies=[2, 8, 32, 128]):
        self.target_movies = target_movies
        self.keep_movies = keep_movies
        self.svd_at=22
        self.svd_algorithm = 'arpack'
        self.ranking_samples = ranking_samples
        
    def fit(self, data):
        self.svd_rank_cache = {}
        self.users = defaultdict(lambda: int(len(self.users)))
        self.movies = defaultdict(lambda: int(len(self.movies)))
        
        self.extract_target(data)
        
        self.movie_reverse = {}
        for movie in self.movies:
            self.movie_reverse[self.movies[movie]] = movie
            
        self.build_svds()
        self.build_top()
        self.build_ranking_dataset()
        self.build_model()
        
    def build_model(self):
        df = copy.deepcopy(self.ranking_dataset)
        user_ids = df['user_id']
        del(df['user_id'])
        target = df['target']
        del(df['target'])
        
        prev_id = None
        group = []
        cnt = 0
        for uid in user_ids:
            if prev_id is not None and uid != prev_id:
                group.append(cnt)
                cnt = 0 
            cnt += 1 
            prev_id = uid
        group.append(cnt)
        self.lgb = LGBMRanker()
        self.lgb.fit(df, target, group=group)
      
    def build_ranking_dataset(self):
        print("building dataset...")
        result = []
        for i in tqdm_notebook(range(self.ranking_samples)):
            while True:
                user = random.randint(0, len(self.train_data) - 1)
                if len(self.train_data[user]) != 0:
                    break
                    
            movies_seq = self.train_data[user]
            movies_seq_hash = self.get_seq_hash(movies_seq)
            target_movies = self.target_data[user]
            candidates_set = self.get_candidates(movies_seq, target_movies, svd_tops=0)
            for candidate in candidates_set:
                doc = self.get_features(movies_seq, movies_seq_hash, candidate)
                doc["user_id"] = user
                doc["target"] = candidate in target_movies
                result.append(doc)
            self.del_from_cache(movies_seq_hash)
        self.ranking_dataset = pd.DataFrame(result)
          
    def del_from_cache(self, key):
        for n in self.keep_movies:
            del(self.svd_rank_cache[(key, n)])

        
    def get_seq_hash(self, seq):
        return mmh3.hash(",".join([str(elem) for elem in seq]))
    
    def get_candidates(self, movies_seq, include_movies=[], random_samples=50, svd_tops=20):
        result = []
        if svd_tops > 0:
            for n in self.keep_movies:
                result += self.get_n_svd_rank(movies_seq, n)[:svd_tops]
                
        for i in range(random_samples):
            result += [random.randint(0, len(self.movies) - 1)]
        
        result += include_movies          
        return(set(result) - set(movies_seq))
    
    def get_n_svd_rank(self, movies_seq, n):
        low_dim = self.get_low_dimension(movies_seq, n)
        model = self.svds[n]
        predictions = model.inverse_transform(low_dim)[0]
        rank = list(reversed(np.argsort(predictions)))
        seq_set = set(movies_seq)
        rank = filter(lambda x: x not in seq_set, rank)
        return list(rank)
        
    def build_top(self):
        print("building tops..")
        movie_cnt = Counter()
        cnt = 0
        for user_id in self.train_data:
            for movie in self.train_data[user_id]:
                movie_cnt[movie] += 1
                cnt += 1
        self.top_list = []
        self.movie_pop = {}
        for (movie, count) in movie_cnt.most_common():
            self.top_list.append(movie)
            self.movie_pop[movie] = count / cnt
        
    def extract_target(self, data):
        self.full_data = {}
        self.train_data = {}
        self.target_data = {}
        print("extracting target...")
        for ext_user_id, group in data.groupby('userId'):
            user_id = self.users[ext_user_id]
            self.full_data[user_id] = [self.movies[int(d.movieId)] for row, d in group.iterrows()]
            self.target_data[user_id] = self.full_data[user_id][-self.target_movies:]
            self.train_data[user_id] = self.full_data[user_id][:-self.target_movies]  
        
    def build_svds(self):
        print("building svds..")
        self.svds = {}
        for n in self.keep_movies:
            self.svds[n] = self.build_svd(n)
            
    def build_svd(self, n):
        print("building svd using {} last movies for user".format(n))  
        rows = []
        cols = []
        
        for user_id in self.train_data:
            user_keep_movies = self.train_data[user_id][-n:]
            cols += user_keep_movies
            rows += [self.users[user_id]] * len(user_keep_movies)
        vals = [1.0]* len(cols)
        interactions_matrix = csr_matrix((vals, (rows, cols)), shape=(len(self.users), len(self.movies)))
        model = TruncatedSVD(n_components=self.svd_at, algorithm=self.svd_algorithm, )
        model.fit(interactions_matrix)
        return model
    
    def get_features(self, movies_seq, movies_seq_hash, movie):
        features = {}
        for n in self.keep_movies:
            features["svd_keep_{}".format(n)] = self.svd_movie_rank(movies_seq, movies_seq_hash, movie, n)
        movie_pop = self.movie_pop.get(movie, 0.0)
        features["movie_pop"] = movie_pop
        features["seq_len"] = len(movies_seq)
        return features
    
    def svd_movie_rank(self, movies_seq, movies_seq_hash, movie, n):
        if (movies_seq_hash, n) in self.svd_rank_cache:
            return self.svd_rank_cache[(movies_seq_hash,n)][movie]
        else:
            low_dimension = self.get_low_dimension(movies_seq, n)
            movies_seq_set = set(movies_seq)
            model = self.svds[n]
            predictions = model.inverse_transform(low_dimension)[0]
            ranks = reversed(np.argsort(predictions))
            ranks = filter(lambda x: x not in movies_seq_set, ranks)
            result = {}
            for movie_rank, movie in enumerate(ranks):
                result[movie] = movie_rank
            for movie in movies_seq_set:
                result[movie] = -1
            self.svd_rank_cache[(movies_seq_hash,n)] = result
            return self.svd_rank_cache[(movies_seq_hash,n)][movie]
            
    def get_low_dimension(self, movies_seq, n):
        n_seq = movies_seq[-n:]
        row = np.zeros(len(self.movies))
        model = self.svds[n]
        for idx in n_seq:
            row[idx] = 1.0
        return model.transform([row])
            
    def get_n_score(self, low_dimension, movie, n):
        model = self.svds[n]
        predictions = model.inverse_transform(low_dimension)[0]
        return predictions[movie]
    
    
    def predict(self, user_id, n_recommendations=10):   
        user = self.users[user_id]
        seq = self.full_data[user]
        seq_hash = self.get_seq_hash(seq)
        candidates = list(self.get_candidates(seq))
        docs = []
        for candidate in candidates:
            features=self.get_features(seq, seq_hash, candidate)
            docs.append(features)
        df = pd.DataFrame(docs, columns=self.ranking_dataset.columns)
        del(df['user_id'])
        del(df['target'])
        predictions = self.lgb.predict(df)
        rec_ids =  list(reversed(np.argsort(predictions)))[:n_recommendations]
        result = []
        for rec_id in rec_ids:
            result.append(self.movie_reverse[candidates[rec_id]])
        self.del_from_cache(seq_hash)
        return result


In [ ]:
x = MLRecommender(ranking_samples=2000)
get_ndcg(x, 0.1)
make_submit(x, "lgbm_svds_submit.csv")

Добавим немного фичей, а именно — максимальную косинусную меру близости для N последних элементов:

In [ ]:
from lightgbm import LGBMRanker
from collections import defaultdict
from scipy.sparse import csr_matrix
from sklearn.decomposition import TruncatedSVD

class MLRecommender(object):
    def __init__(self, target_movies = 3, 
                 svd_at=22,
                 ranking_samples=150,
                 keep_movies=[2, 8, 32, 128], 
                 sims_at=[1, 2 , 8, 32, 128]):
        self.target_movies = target_movies
        self.keep_movies = keep_movies
        self.svd_at=22
        self.svd_algorithm = 'arpack'
        self.sims_at = sims_at
        self.ranking_samples = ranking_samples
        
    def fit(self, data):
        self.svd_rank_cache = {}
        self.users = defaultdict(lambda: int(len(self.users)))
        self.movies = defaultdict(lambda: int(len(self.movies)))
        
        self.extract_target(data)
        
        self.movie_reverse = {}
        for movie in self.movies:
            self.movie_reverse[self.movies[movie]] = movie
            
        self.build_svds()
        self.build_top()
        self.build_movies_sims()
        self.build_ranking_dataset()
        self.build_model()
        

    
    def build_model(self):
        df = copy.deepcopy(self.ranking_dataset)
        user_ids = df['user_id']
        del(df['user_id'])
        target = df['target']
        del(df['target'])
        
        prev_id = None
        group = []
        cnt = 0
        for uid in user_ids:
            if prev_id is not None and uid != prev_id:
                group.append(cnt)
                cnt = 0 
            cnt += 1 
            prev_id = uid
        group.append(cnt)
        self.lgb = LGBMRanker()
        self.lgb.fit(df, target, group=group)
      
    def build_ranking_dataset(self):
        print("building dataset...")
        result = []
        for i in tqdm_notebook(range(self.ranking_samples)):
            while True:
                user = random.randint(0, len(self.train_data) - 1)
                if len(self.train_data[user]) != 0:
                    break
                    
            movies_seq = self.train_data[user]
            movies_seq_hash = self.get_seq_hash(movies_seq)
            target_movies = self.target_data[user]
            candidates_set = self.get_candidates(movies_seq, target_movies, svd_tops=0)
            for candidate in candidates_set:
                doc = self.get_features(movies_seq, movies_seq_hash, candidate)
                doc["user_id"] = user
                doc["target"] = candidate in target_movies
                result.append(doc)
            self.del_from_cache(movies_seq_hash)
        self.ranking_dataset = pd.DataFrame(result)
        
    
    def build_movies_sims(self):
        print("building movie sims...")
        n_last_movies = 30
        group_cnt = len(self.train_data)
        movie_cnt = Counter()
        pair_cnt = Counter()
        for group in self.train_data.values():
            for movie1 in set(group[-n_last_movies:]):
                movie_cnt[movie1] += 1
                for movie2 in set(group[-n_last_movies:]):
                    if movie1 == movie2:
                        continue
                    pair_cnt[(movie1, movie2)] += 1
        result = {}
        for (movie1, movie2) in pair_cnt:
            pair_prob = pair_cnt[(movie1, movie2)] / group_cnt
            movie1_prob = movie_cnt[movie1] / group_cnt
            movie2_prob = movie_cnt[movie2] / group_cnt
            result[(movie1, movie2)] = pair_prob ** 2 / (movie1_prob * movie2_prob)
        self.movies_sims = result
    
    def max_sim(self, movies_seq, movie, n_last_movies):
        result = 0.0
        for m in movies_seq[-n_last_movies:]:
            result = max(result, self.movies_sims.get((m, movie), 0.0))
        return result
          
    def del_from_cache(self, key):
        for n in self.keep_movies:
            del(self.svd_rank_cache[(key, n)])

        
    def get_seq_hash(self, seq):
        return mmh3.hash(",".join([str(elem) for elem in seq]))
    
    def get_candidates(self, movies_seq, include_movies=[], random_samples=50, svd_tops=20):
        result = []
        if svd_tops > 0:
            for n in self.keep_movies:
                result += self.get_n_svd_rank(movies_seq, n)[:svd_tops]
                
        for i in range(random_samples):
            result += [random.randint(0, len(self.movies) - 1)]
        
        result += include_movies          
        return(set(result) - set(movies_seq))
    
    def get_n_svd_rank(self, movies_seq, n):
        low_dim = self.get_low_dimension(movies_seq, n)
        model = self.svds[n]
        predictions = model.inverse_transform(low_dim)[0]
        rank = list(reversed(np.argsort(predictions)))
        seq_set = set(movies_seq)
        rank = filter(lambda x: x not in seq_set, rank)
        return list(rank)
        
    def build_top(self):
        print("building tops..")
        movie_cnt = Counter()
        cnt = 0
        for user_id in self.train_data:
            for movie in self.train_data[user_id]:
                movie_cnt[movie] += 1
                cnt += 1
        self.top_list = []
        self.movie_pop = {}
        for (movie, count) in movie_cnt.most_common():
            self.top_list.append(movie)
            self.movie_pop[movie] = count / cnt
        
    def extract_target(self, data):
        self.full_data = {}
        self.train_data = {}
        self.target_data = {}
        print("extracting target...")
        for ext_user_id, group in data.groupby('userId'):
            user_id = self.users[ext_user_id]
            self.full_data[user_id] = [self.movies[int(d.movieId)] for row, d in group.iterrows()]
            self.target_data[user_id] = self.full_data[user_id][-self.target_movies:]
            self.train_data[user_id] = self.full_data[user_id][:-self.target_movies]  
        
    def build_svds(self):
        print("building svds..")
        self.svds = {}
        for n in self.keep_movies:
            self.svds[n] = self.build_svd(n)
            
    def build_svd(self, n):
        print("building svd using {} last movies for user".format(n))  
        rows = []
        cols = []
        
        for user_id in self.train_data:
            user_keep_movies = self.train_data[user_id][-n:]
            cols += user_keep_movies
            rows += [self.users[user_id]] * len(user_keep_movies)
        vals = [1.0]* len(cols)
        interactions_matrix = csr_matrix((vals, (rows, cols)), shape=(len(self.users), len(self.movies)))
        model = TruncatedSVD(n_components=self.svd_at, algorithm=self.svd_algorithm, )
        model.fit(interactions_matrix)
        return model
    
    def get_features(self, movies_seq, movies_seq_hash, movie):
        features = {}
        for n in self.keep_movies:
            features["svd_keep_{}".format(n)] = self.svd_movie_rank(movies_seq, movies_seq_hash, movie, n)
        
        for n in self.sims_at:
            features["cos_sim_{}".format(n)] = self.max_sim(movies_seq, movie, n)
        movie_pop = self.movie_pop.get(movie, 0.0)
        features["movie_pop"] = movie_pop
        features["seq_len"] = len(movies_seq)
        return features
    
    def svd_movie_rank(self, movies_seq, movies_seq_hash, movie, n):
        if (movies_seq_hash, n) in self.svd_rank_cache:
            return self.svd_rank_cache[(movies_seq_hash,n)][movie]
        else:
            low_dimension = self.get_low_dimension(movies_seq, n)
            movies_seq_set = set(movies_seq)
            model = self.svds[n]
            predictions = model.inverse_transform(low_dimension)[0]
            ranks = reversed(np.argsort(predictions))
            ranks = filter(lambda x: x not in movies_seq_set, ranks)
            result = {}
            for movie_rank, movie in enumerate(ranks):
                result[movie] = movie_rank
            for movie in movies_seq_set:
                result[movie] = -1
            self.svd_rank_cache[(movies_seq_hash,n)] = result
            return self.svd_rank_cache[(movies_seq_hash,n)][movie]
            
    def get_low_dimension(self, movies_seq, n):
        n_seq = movies_seq[-n:]
        row = np.zeros(len(self.movies))
        model = self.svds[n]
        for idx in n_seq:
            row[idx] = 1.0
        return model.transform([row])
            
    def get_n_score(self, low_dimension, movie, n):
        model = self.svds[n]
        predictions = model.inverse_transform(low_dimension)[0]
        return predictions[movie]
    
    
    def predict(self, user_id, n_recommendations=10):   
        user = self.users[user_id]
        seq = self.full_data[user]
        seq_hash = self.get_seq_hash(seq)
        candidates = list(self.get_candidates(seq))
        docs = []
        for candidate in candidates:
            features=self.get_features(seq, seq_hash, candidate)
            docs.append(features)
        df = pd.DataFrame(docs, columns=self.ranking_dataset.columns)
        del(df['user_id'])
        del(df['target'])
        predictions = self.lgb.predict(df)
        rec_ids =  list(reversed(np.argsort(predictions)))[:n_recommendations]
        result = []
        for rec_id in rec_ids:
            result.append(self.movie_reverse[candidates[rec_id]])
        self.del_from_cache(seq_hash)
        return result


In [ ]:
x = MLRecommender(ranking_samples=2000)
get_ndcg(x, 0.1)
make_submit(x, "lgbm_svds_sims_submit.csv")

Данный подход набирает 0.06856 на kaggle, но наверняка можно набрать и больше!

Можно выполнить поиск гиперпараметров модели Lightgbm, поиграться со значениями N, которые мы сохраняем. 

Также можно добавить дополнительных фичей в наш ML-рекоммендер. Фичи теперь ограничены только вашей фантазией, например: 

* Кратковременная популярность фильма
* Средняя косинусная близость
* Близость фильмов по тэгам
* Близость фильмов по названиям


